## Download necessary libraries 

In [0]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import zipfile
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

## Download celebA dataset

In [2]:
!mkdir data_faces && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip 

with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

dataDir = 'data_faces/img_align_celeba'
img_list = os.listdir(dataDir)
print(len(img_list))

--2020-02-25 23:35:56--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.120.8
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.120.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1443490838 (1.3G) [application/zip]
Saving to: ‘celeba.zip’

celeba.zip          100%[===================>]   1.34G  21.1MB/s    in 67s     

2020-02-25 23:37:09 (20.6 MB/s) - ‘celeba.zip’ saved [1443490838/1443490838]

202599


In [0]:
# Create the dataset
dataset = dset.ImageFolder(root='./data_faces',
                           transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.CenterCrop(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

## Useful variable definitions are provided

In [0]:
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector (i.e. size of generator input)
nz = 100
# size of feature maps in generator
ngf = 64
# Size of feature maps in discriminator
ndf = 64

## Implement generator & discriminator



In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

## Define the model and hyperparameters -- Do **NOT** change!
GAN training is very sensitive to hyperparameter setting.
Here, we provide the set of hyperparameters that the authors in their original implementation. Please do not change any values. Otherwise, we cannot guarantee that your GAN will converge.

In [0]:
# Number of training epochs
num_epochs = 2
# Batch size during training
batch_size = 128
# Learning rate for optimizers
lr = 0.0002
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
beta2 = 0.999

# Setup Adam optimizers for both G and D
netD = Discriminator()
netG = Generator()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))

criterion = torch.nn.BCELoss()

## Train Loop

In [0]:
# Training Loop
use_cuda = True
device = torch.device('cuda')

# Lists to keep track of progress
img_list = []
D_loss_store = []
G_loss_store = []

netD = netD.to(device)
netG = netG.to(device)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    G_losses = []
    D_losses = []
        # For each batch in the dataloader
    for i, (images, _) in enumerate(dataloader):

        # Train discriminator with real data
        mini_batch = images.size()[0]
        images_real = images.to(device)
        y_real = torch.ones(mini_batch).to(device)
        y_fake = torch.zeros(mini_batch).to(device)

        # Forward pass real batch through D
        d_output_real = netD(images_real).squeeze()
        d_loss_real = criterion(d_output_real, y_real)

        # Train discriminator with fake data
        noise = torch.randn(mini_batch, nz, 1, 1).to(device)
        images_fake = netG(noise)

        # Forward pass fake batch through D
        d_output_fake = netD(images_fake).squeeze()
        d_loss_fake = criterion(d_output_fake, y_fake)

        # Backprop
        d_loss = d_loss_real + d_loss_fake
        netD.zero_grad()
        d_loss.backward()
        optimizerD.step()

        # Train the generator
        noise = torch.randn(mini_batch, nz, 1, 1).to(device)
        images_fake = netG(noise)

        d_output_fake = netD(images_fake).squeeze()
        g_loss = criterion(d_output_fake, y_real)

        # Backprop
        netD.zero_grad()
        netG.zero_grad()
        g_loss.backward()
        optimizerG.step()

        D_losses.append(d_loss.data)
        G_losses.append(g_loss.data)
    
    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))
    
    D_loss_store.append(D_avg_loss.data)
    G_loss_store.append(G_avg_loss.data)
    
    print(f'Epoch {epoch+1}/{num_epochs}, D_loss: {D_avg_loss.data}, G_loss: {G_avg_loss.data}')
    
    if ((epoch == num_epochs-1) and i == len(dataloader) - 1):
        with torch.no_grad():
            fixed_noise = torch.randn(64, nz, 1, 1, device=device)
            fake = netG(fixed_noise).detach().cpu()
        img_list.append(vutils.make_grid(fake, padding = 2, normalize=True))

## Compare real and fake images

In [0]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()